In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from torch.amp import GradScaler
import os

from Training.Data.Modules.custom_loader import CustomLoader 
from Utils.model_utils import *; 
from Utils.configs import conf
from XAI_Method.rand_samples import RandomSamples

In [ ]:
if True:
    
    config = conf['simpleDNNTMnist']
    def init_model(rand=True):
        return get_model_architecture(config['model_name'], config['model_layers'] + [config["num_classes"]], rand)

    model = init_model()   
    data_loader = CustomLoader(config["dataset"], True, config["batch_size"], shuffle_test=True)
    train_loader, test_loader = data_loader.train_load, data_loader.evalu_load

    optimizer = optim.SGD(model.parameters(), lr=1)
    loss = nn.CrossEntropyLoss()
    scaler = GradScaler(enabled=True)

    rand_samples = RandomSamples(model, test_loader, 6, 10)
    R = rand_samples.artificial_step()

    X, labels, const_model = rand_samples.X, rand_samples.lbls, rand_samples.const_model
    device = torch.device('cuda') 
    model = model.to(device)
    model.train()

In [ ]:
os.makedirs(os.getcwd() + f"/Results/Experiments/00_artificial_step", exist_ok=True)
for i in range(4):

    for j, r in enumerate([R[i, :], X[i, :]]):
        title = f'sample_{i}' if j==1 else f'relevance_{i}'
        plt.imshow(r.reshape(28, 28)); plt.axis('off')
        plt.savefig(os.getcwd() + f"/Results/Experiments/00_artificial_step/{title}.pdf", 
                        bbox_inches='tight', format='pdf')
        plt.close()
